In [19]:
import re
from functools import lru_cache
import pymorphy2
import numpy as np
import pandas as pd
from annoy import AnnoyIndex
from gensim.models.word2vec import Word2Vec

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from gensim.models.word2vec import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [21]:
MORPH = pymorphy2.MorphAnalyzer()

@lru_cache(maxsize=100000)
def get_normal_form (i):
    return MORPH.normal_forms(i)[0]


def normalize_text(text):
    text = text[:500]
    normalized = [ get_normal_form(word)  for word in re.findall('[a-zA-Zа-яА-Я]{3,}', text)] # [a-zA-Zа-яА-Я]{3,}|\d+
    return ' '.join([word for word in normalized if word not in []])

stop_words = [get_normal_form(i) for i in []] # стоп-слова

# Классификатор проганье-не проганье

In [22]:
def normalization_vectorization(phrases, size, filename_vec):
    norm_phrases = phrases.apply(normalize_text)
    sentences = norm_phrases.str.split()
    
    model = Word2Vec(sentences, size=size, workers=2, iter=3) # последние два
#    filename2 = name_vect
    joblib.dump(model, filename_vec)
    
    w2v = dict(zip(model.wv.index2word, model.wv.syn0)) # словарь из слов
    
#    embed_matrix = []
#    for i in w2v:
#        embed_matrix.append(w2v[i])
#    embed_matrix = np.vstack(embed_matrix)
    
    data_storage = list(norm_phrases.values)

    
    data_storage_norm = {} # хранит вектора для фраз
    for i in range(len(data_storage)):
        vec = np.zeros(size)
        for word in data_storage[i].split(' '):
            if word in model:
                vec+=model[word] 
        vec /= len(data_storage[i].split(' '))
        data_storage_norm[i] = vec
        
    lst =[]
    for v in data_storage_norm.values():
        lst.append(v)
    q = np.array(lst)
    
    return [data_storage, data_storage_norm , q]
    

In [23]:
def normalization_vectorization_2(phrases, size, filename_vec, filename_d):
    norm_phrases = phrases.apply(normalize_text)
    sentences = norm_phrases.str.split()
    
    model = Word2Vec(sentences, size=size, workers=2, iter=3) # последние два
#    filename2 = name_vect
    joblib.dump(model, filename_vec)
    
    w2v = dict(zip(model.wv.index2word, model.wv.syn0)) # словарь из слов
    
#    embed_matrix = []
#    for i in w2v:
#        embed_matrix.append(w2v[i])
#    embed_matrix = np.vstack(embed_matrix)
    
    data_storage = list(norm_phrases.values)

    tfidf = TfidfVectorizer()
    X = tfidf.fit_transform(data_storage)
    idf = tfidf.idf_
    d = dict(zip(tfidf.get_feature_names(), idf))
    
    with open(filename_d, 'wb') as f:
        pickle.dump(d, f)
    
    data_storage_norm = {} # хранит вектора для фраз
    for i in range(len(data_storage)):
        vec = np.zeros(size)
        for word in data_storage[i].split(' '):
            if word in model and word in d:
                vec += model[word] * d[word]
        vec /= len(data_storage[i].split(' '))
        data_storage_norm[i] = vec
        
    lst =[]
    for v in data_storage_norm.values():
        lst.append(v)
    q = np.array(lst)
    
    return [data_storage, data_storage_norm, q]
    

In [24]:
from gensim.models.word2vec import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

def fit(phrases, target_values, size, filename_class, filename_vec): # принимает  столбец фраз и таргетов
    
    q = normalization_vectorization(phrases, size, filename_vec)[2]
    
    clf = LogisticRegression()
    clf.fit(q,target_values)
    
#    filename1 = name_classif
    joblib.dump(clf, filename_class)
    return None
   
    
    #return что-то важное

In [25]:
from gensim.models.word2vec import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

def fit_2(phrases, target_values, size, filename_class, filename_vec, filename_d): # принимает  столбец фраз и таргетов
    
    q = normalization_vectorization_2(phrases, size, filename_vec, filename_d)[2]
    
    clf = LogisticRegression()
    clf.fit(q,target_values)
    
#    filename1 = name_classif
    joblib.dump(clf, filename_class)
    return None
   
    
    #return что-то важное

In [26]:
def vectorize_one(zapros, size, w2v):
    zapros = normalize_text(zapros)
    seq = np.zeros(size)
    for word in zapros.split():
        if word in w2v:
            seq += w2v[word]
    seq /= len(zapros.split())
    return seq.reshape(1,-1)

In [176]:
def vectorize_one_2(zapros, size, w2v, d):
    zapros = normalize_text(zapros)
    seq = np.zeros(size)
    for word in zapros.split():
        if word in w2v and word in d:
            seq += w2v[word] * d[word]
    seq /= len(zapros.split())
    return seq.reshape(1,-1)

In [28]:
def pred_zapros(classif_model, zapros, size, w2v):
    vect_zapros = vectorize_one(zapros, size, w2v)
    if np.array_equal(vect_zapros, np.zeros(size).reshape(1,-1)):
        return -1
    y_hat = classif_model.predict(vectorize_one(zapros, size, w2v))
    return y_hat[0]

In [29]:
def pred_zapros_2(classif_model, zapros, size, w2v, d):
    vect_zapros = vectorize_one_2(zapros, size, w2v, d)
    if np.array_equal(vect_zapros, np.zeros(size).reshape(1,-1)):
        return -1
    y_hat = classif_model.predict(vectorize_one_2(zapros, size, w2v, d))
    return y_hat[0]

# DOING

In [30]:
name_classif = 'logreg_for_prog.sav'
name_classif_2 = 'logreg_for_prog_2.sav'
name_vect = 'word2vec_for_prog.sav'
name_vect_2 = 'word2vec_for_prog_2.sav'
name_tfidf = "dict_tfidf.pkl"
size = 100

# Импортирование моих данных

In [219]:
stack_phrases = pd.read_csv('stack_phrases.csv')
quest_ans_from_mail = pd.read_csv('quest_ans_from_mail.csv')

# Обработка данных

In [220]:
stack_phrases.shape

(256042, 2)

In [221]:
quest_ans_from_mail.shape

(1163270, 2)

In [222]:
stack_phrases.head()

,sources,phrases
0,Stackoverflow,Как из скрипта на Питоне послать письмо с влож...
1,Stackoverflow,Как сохранить и восстановить базу данных Postg...
2,Stackoverflow,Как найти файл по имени в папках командой из т...
3,Stackoverflow,Как работать с svn через HTTP прокси на Ubuntu?
4,Stackoverflow,Как подсчитать строки в файле из терминала в U...


In [223]:
quest_ans_from_mail.head()

,questions,answers
0,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,хомячка....
1,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :).
2,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали..."
3,Эбу в двенашке называется Итэлма что за эбу? .,ЭБУ — электронный блок управления двигателем а...
4,академия вампиров. сколько на даный момент час...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть..."


In [224]:
quest_ans_from_mail["classes"] = 0
stack_phrases["classes"] = 1

In [225]:
quest_ans_from_mail.head()

,questions,answers,classes
0,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,хомячка....,0
1,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :).,0
2,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали...",0
3,Эбу в двенашке называется Итэлма что за эбу? .,ЭБУ — электронный блок управления двигателем а...,0
4,академия вампиров. сколько на даный момент час...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть...",0


In [226]:
stack_phrases.head()

,sources,phrases,classes
0,Stackoverflow,Как из скрипта на Питоне послать письмо с влож...,1
1,Stackoverflow,Как сохранить и восстановить базу данных Postg...,1
2,Stackoverflow,Как найти файл по имени в папках командой из т...,1
3,Stackoverflow,Как работать с svn через HTTP прокси на Ubuntu?,1
4,Stackoverflow,Как подсчитать строки в файле из терминала в U...,1


In [227]:
quest_ans_from_mail = quest_ans_from_mail.drop(columns="answers")
quest_ans_from_mail = quest_ans_from_mail.rename(columns={"questions": "phrases"})
stack_phrases = stack_phrases.drop(columns="sources")

In [228]:
quest_ans_from_mail.head()

,phrases,classes
0,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,0
1,Как парни относятся к цветным линзам? Если у д...,0
2,"Что делать, сегодня нашёл 2 миллиона рублей? .",0
3,Эбу в двенашке называется Итэлма что за эбу? .,0
4,академия вампиров. сколько на даный момент час...,0


In [229]:
stack_phrases.head()

,phrases,classes
0,Как из скрипта на Питоне послать письмо с влож...,1
1,Как сохранить и восстановить базу данных Postg...,1
2,Как найти файл по имени в папках командой из т...,1
3,Как работать с svn через HTTP прокси на Ubuntu?,1
4,Как подсчитать строки в файле из терминала в U...,1


In [230]:
df_data_for_prog_classification = quest_ans_from_mail[:256042].append(stack_phrases, ignore_index=True)

In [231]:
df_data_for_prog_classification.tail()

,phrases,classes
512079,Django 2.1.7 ! Шаблон не выводит переменную,1
512080,"DBNavigator – кнопка ""refresh"" не доступна",1
512081,Как настроить URl каталога в меню Bitrix?,1
512082,JWT Token не работает с ролями,1
512083,Запуск нескольких задач (Task) из Task и ожида...,1


# Сохранение данных

In [232]:
df_data_for_prog_classification.to_csv('data_for_prog_classification.csv', index=None)

# Загрузим csv

In [31]:
df_data_for_prog_classification = pd.read_csv('data_for_prog_classification.csv')

In [32]:
df_data_for_prog_classification.sample(10)

,phrases,classes
220333,между городами 730 км. поезд шел 5 часов со ск...,0
403956,Адаптивная верстка psd,1
276245,JOIN выборка с несоответствующим id из родств...,1
374933,Как определить размер структуры ?C#,1
25585,Сколодина.,0
408655,Почему не удаляются посты с картинками в блоге?,1
297692,Как выбрать всё между кавычками?,1
218373,"Закрыл процесс explorer.exe, без перезагрузки ...",0
3078,Что такое время!!!? Нет дня ближе завтрешнего ...,0
381242,Распарсить Json файл с несколькими массивами,1


# Работа с моделью (обучение, предикт)

In [15]:
fit(df_data_for_prog_classification['phrases'], 
    df_data_for_prog_classification['classes'], 
    size, 
    name_classif,
    name_vect)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
fit_2(df_data_for_prog_classification['phrases'], 
    df_data_for_prog_classification['classes'], 
    size, 
    name_classif_2,
    name_vect_2, 
    name_tfidf)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [37]:
#classif_model = joblib.load(name_classif)
#loaded_model = joblib.load(name_vect)
classif_model_2 = joblib.load(name_classif_2)
loaded_model_2 = joblib.load(name_vect_2)
#w2v = dict(zip(loaded_model.wv.index2word, loaded_model.wv.syn0))
w2v_2 = dict(zip(loaded_model_2.wv.index2word, loaded_model_2.wv.syn0)) # продуюблировать пришлось
with open(name_tfidf, 'rb') as f:
    d = pickle.load(f)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [39]:
with open("w2v_2_for_prog.pkl", 'wb') as f:
    pickle.dump(w2v_2, f)

In [40]:
zapros = input()

массив случайных чисел


In [19]:
pred_zapros(classif_model, zapros, size, w2v)

0

In [42]:
pred_zapros_2(classif_model_2, zapros, size, w2v_2, d)

1

# Это анной (пока не до конца покрутил) - он принимает на вход data_storage_norm и data_storage

# Загрузим вопросы со stackoverflow

In [43]:
df_stack_questions = pd.read_csv('stack_questions.csv')
df_stack_answers = pd.read_csv('stack_answers.csv')

In [44]:
df_stack_questions.head()

,ids_questions,ids_accepted_answer,titles,tags
0,1,2,Как из скрипта на Питоне послать письмо с влож...,python
1,3,4,Как сохранить и восстановить базу данных Postg...,no_tag
2,5,6,Как найти файл по имени в папках командой из т...,no_tag
3,7,8,Как работать с svn через HTTP прокси на Ubuntu?,no_tag
4,9,10,Как подсчитать строки в файле из терминала в U...,no_tag


In [45]:
df_stack_answers.head()

,ids_answers,ids_parent,answers
0,2,1,from smtplib import SMTP_SSL\nfrom email.MIMEM...
1,4,3,Для сохранения кластера базы данных:\n\npg_dum...
2,6,5,Для подобного поиска подходит команда find.\n\...
3,8,7,Нужно прописать параметры прокси в настройках ...
4,10,9,"Команда wc выводит количество переводов строк,..."


In [20]:
df_stack_questions[df_stack_questions["ids_questions"]==3]["titles"].values[0]

'Как сохранить и восстановить базу данных PostgreSQL?'

In [46]:
# filename_stack_vec = "stackoverflow_vec.sav"
filename_stack_vec_2 = "stackoverflow_vec_2.sav"
filename_tfidf_for_annoy = "tfidf_for_annoy_for_stack.pkl"

In [25]:
def vectorize_for_annoy_for_stack(data_col, size, filename_vec):
    data_storage_vec = normalization_vectorization(data_col, size, filename_vec)[2]
    data_storage_norm = {}
    for i in range(len(data_storage_vec)):
        key = df_stack_questions["ids_questions"][i]
        data_storage_norm[key] = data_storage_vec[i]
    #    data_storage_norm[key] = [df_stack_questions["titles"][i], data_storage_vec[i]]
    return data_storage_norm

In [26]:
def vectorize_for_annoy(id_data, phrases_data, size, filename_vec):
    data_storage_vec = normalization_vectorization(phrases_data, size, filename_vec)[2]
    data_storage_norm = {}
    for i in range(len(data_storage_vec)):
        key = id_data[i]
        data_storage_norm[key] = data_storage_vec[i]
    #    data_storage_norm[key] = [df_stack_questions["titles"][i], data_storage_vec[i]]
    return data_storage_norm

In [47]:
def vectorize_for_annoy_2(id_data, phrases_data, size, filename_vec, d):
    data_storage_vec = normalization_vectorization_2(phrases_data, size, filename_vec, d)[2]
    data_storage_norm = {}
    for i in range(len(data_storage_vec)):
        key = id_data[i]
        data_storage_norm[key] = data_storage_vec[i]
    #    data_storage_norm[key] = [df_stack_questions["titles"][i], data_storage_vec[i]]
    return data_storage_norm

In [48]:
def build_annoy(data_storage_norm, NUM_TREES, VEC_SIZE_EMB):
    #NUM_TREES = 7
    #VEC_SIZE_EMB = 100
    #VEC_SIZE_EMB = 300

    counter = 0
    map_id_2_prod_hash = {}
    index_title_emb = AnnoyIndex(VEC_SIZE_EMB)

    print("Build annoy base")
    for prod_hash in data_storage_norm:
        title_vec = data_storage_norm[prod_hash] # Вытаскиваем вектор

        index_title_emb.add_item(counter, title_vec) # Кладем в анной

        map_id_2_prod_hash[counter] = prod_hash # сохраняем мапу - (id в анное -> продукт_id)

        counter += 1
        if counter % 10000 == 1:
            print("computed for %d" % counter)

    index_title_emb.build(NUM_TREES)
    
    index_title_emb.save("annoy_for_mail.ann")
    with open("map_id_for_mail.pkl", 'wb') as f:
        pickle.dump(map_id_2_prod_hash, f)
    
    print("builded")
    return {'index_title_emb': index_title_emb, 'map_id_2_prod_hash': map_id_2_prod_hash}

In [29]:
data_storage_norm = vectorize_for_annoy(df_stack_questions["ids_questions"], df_stack_questions["titles"], 
                                        300, filename_stack_vec)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [49]:
data_storage_norm_2 = vectorize_for_annoy_2(df_stack_questions["ids_questions"], df_stack_questions["titles"], 
                                        300, filename_stack_vec_2, filename_tfidf_for_annoy)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [50]:
with open("stack_annoy_norm.pkl", 'wb') as f:
    pickle.dump(data_storage_norm_2, f)

In [141]:
data_storage_norm_2

{1: array([-1.73543987e+00, -1.98606896e+00, -5.33230222e+00,  5.48137951e+00,
        -2.62468433e+00, -3.12843696e+00,  6.17690676e+00, -7.15659098e+00,
        -3.40729804e+00, -2.95266805e+00, -1.29308585e+00, -8.84670790e-01,
        -1.12663892e+00, -2.59009200e+00, -6.85341595e+00, -5.81958551e+00,
         3.09366371e+00,  3.34003758e+00, -1.64067957e+00,  1.68287146e+00,
         3.22100313e+00,  4.08763918e+00, -2.50743330e+00, -5.04697898e+00,
         2.88495732e+00, -1.68517183e+00, -1.09179965e+00,  9.43390735e-01,
        -4.28144841e+00, -7.81427906e+00,  1.58228447e+00,  2.89327487e+00,
        -2.48059901e-03, -7.41047130e-01, -2.00457976e+00,  5.15073885e+00,
        -1.29404015e+00,  1.34929782e+00, -2.64153149e+00, -2.41007133e+00,
         3.37853128e+00, -5.66770580e+00,  2.15048958e+00, -3.08553509e-01,
         5.78477713e+00,  5.08508227e-01, -2.29055850e+00, -2.22925401e+00,
        -3.09339571e+00, -1.56464960e+00,  2.33706253e+00,  1.69881752e+00,
         

In [52]:
NUM_TREES = 20
VEC_SIZE_EMB = 300
#index_title_emb = build_annoy(data_storage_norm, NUM_TREES, VEC_SIZE_EMB)

In [53]:
index_title_emb_2 = build_annoy(data_storage_norm_2, NUM_TREES, VEC_SIZE_EMB)

Build annoy base
computed for 1
computed for 10001
computed for 20001
computed for 30001
computed for 40001
computed for 50001
computed for 60001
computed for 70001
computed for 80001
computed for 90001
computed for 100001
computed for 110001
computed for 120001
computed for 130001
computed for 140001
computed for 150001
computed for 160001
computed for 170001
computed for 180001
computed for 190001
computed for 200001
computed for 210001
computed for 220001
computed for 230001
computed for 240001
computed for 250001
builded


In [33]:
stack_vec_model = joblib.load(filename_stack_vec)
w2v = dict(zip(stack_vec_model.wv.index2word, stack_vec_model.wv.syn0))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [54]:
stack_vec_model_2 = joblib.load(filename_stack_vec_2)
w2v_2 = dict(zip(stack_vec_model_2.wv.index2word, stack_vec_model_2.wv.syn0))
with open(filename_tfidf_for_annoy, 'rb') as f:
    d = pickle.load(f)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [55]:
with open("w2v_2_for_stack.pkl", 'wb') as f:
        pickle.dump(w2v_2, f)

In [56]:
zapros = input()

подсчитать строки в файле ubuntu


In [114]:
zapros_vec = vectorize_one(zapros, 300, w2v)

In [57]:
zapros_vec_2 = vectorize_one_2(zapros, 300, w2v_2, d)

In [167]:
import itertools

#title_id = list(data_storage_norm.keys())[np.random.randint(10000)]

#zapros = df_stack_questions[df_stack_questions["ids_questions"]==title_id]["titles"].values[0]
#annoy_res = list(index_title_emb.get_nns_by_vector(zapros_vec.T, 10, include_distances=True))

def answer_for_stack(annoy_res, map_id_2_prod_hash, df_questions, df_answers, count_quest=1, count_ans=1):
    res = []
#    print('Запрос:', zapros)
#    print('Векторок:', data_storage_norm[title_id][:20], '\n')
#    print("===" * 20)
#    print("===" * 20 + '\n')
    #annoy_res = list(index_title_emb.get_nns_by_vector(data_storage_norm[title_id], 13, include_distances=True))
#    annoy_res = list(index_title_emb.get_nns_by_vector(zapros_vec.T, 10, include_distances=True))

    #print('\n\nСоседи:')

    for annoy_id, annoy_sim in itertools.islice(zip(*annoy_res), count_quest):
        image_id = map_id_2_prod_hash[annoy_id]    
        quest = df_stack_questions[df_stack_questions["ids_questions"]==image_id]["titles"].values[0]
    #    print(ans, 1 - annoy_sim ** 2 / 2)
        #print("Похожий вопрос:")
        #print(quest, '\n')
        #print("___" * 20 + '\n')
        accepted_id = df_stack_questions[df_stack_questions["ids_questions"]==image_id] \
                                        ["ids_accepted_answer"].values[0]
        if accepted_id == -1:
            ans = df_stack_answers[df_stack_answers["ids_parent"] == image_id]["answers"]
            id_ans = df_stack_answers[df_stack_answers["ids_parent"] == image_id]["ids_answers"]
            if len(ans) > 0:
                #print("Попробуй это:")
                z = 0
                for i in id_ans.values:
                    res = i
                    z += 1
                    if z == count_ans:
                        break
                z = 0
                for i in ans.values:
                    #print(i)
                    z += 1
                    if z == count_ans:
                        break
                    #print("___" * 20)
            else:
                #print("Нет ответа!!!\n")
                res = -1
        else:
            #print("Найден правильный ответ: \n")
            ans = df_stack_answers[df_stack_answers["ids_answers"] == accepted_id]["answers"]
            id_ans = df_stack_answers[df_stack_answers["ids_answers"] == accepted_id]["ids_answers"]
            res = id_ans.values[0]
            # print(ans.values[0])
        #print('\n', "===" * 20, '\n')
        return res

In [59]:
annoy_res = list(index_title_emb['index_title_emb'].get_nns_by_vector(zapros_vec.T, 10, include_distances=True))

NameError: name 'index_title_emb' is not defined

In [117]:
answer_for_stack(annoy_res, index_title_emb['map_id_2_prod_hash'], df_stack_questions, df_stack_answers)

Похожий вопрос:
Delphi XE2: «Лишние» строки в исполняемом файле 

____________________________________________________________

Попробуй это:
Сомневаюсь, что эти данные лишние.
Delphi при сборке обычно прикомпилирует некоторые "свои" модули к программе. Это можно отключить в настройках компилятора. Но, тогда придётся всегда "таскать" с программой все необходимые компоненты в виде .bpl включая те же кнопки и т.д. (смотря какие модули отключите)
Вообще, релиз - это максимум что может дать компилятор.
Большей экономии размера можно достичь, например, как уже говорилось, вынеся всё из экзешника ("таская" .bpl) или отказавшись от VCL (FireMonkey) в пользу чистого WinAPI (если это действительно оправдано).

____________________________________________________________
1) В dpr файле между ключевым словом program и разделом uses напишите:
program XXX;

{$WEAKLINKRTTI ON}        // если вам не нужны новые возможности RTTI!
{$RTTI EXPLICIT METHODS([]) FIELDS([]) PROPERTIES([])}    // если вам не

[456436, 456469]

In [60]:
annoy_res_2 = list(index_title_emb_2['index_title_emb']
                   .get_nns_by_vector(zapros_vec_2.T, 10, include_distances=True))

In [61]:
answer_for_stack(annoy_res_2, index_title_emb_2['map_id_2_prod_hash'], df_stack_questions, df_stack_answers)

Похожий вопрос:
Как подсчитать строки в файле из терминала в Ubuntu? 

____________________________________________________________

Найден правильный ответ: 

Команда wc выводит количество переводов строк, слов и байтов в текстовом файле.
Например.

echo "Первая сторока" > текст
echo "Вторая сторока" >> текст
wc -l текст
2 текст





[10]

# Ответы mail.ru

In [156]:
import itertools

def answer_for_mail(annoy_res, map_id_2_prod_hash, df_quest_ans_mail_ans, count_quest=1, count_ans=3):
    res = []
    for annoy_id, annoy_sim in itertools.islice(zip(*annoy_res), count_quest):
        image_id = map_id_2_prod_hash[annoy_id]    
        quest = df_quest_ans_mail_ans[df_quest_ans_mail_ans["id"]==image_id]["questions"].values[0]
        ans = df_quest_ans_mail_ans[df_quest_ans_mail_ans["id"]==image_id]["answers"].values[0]
        res = df_quest_ans_mail_ans[df_quest_ans_mail_ans["id"]==image_id]["id"].values[0]
        #print("Похожий вопрос:")
        #print(quest, '\n')
        #print("Ответ: ")
        #print(ans, '\n')
        return res

In [65]:
df_quest_ans_mail_ans = pd.read_csv('quest_ans_from_mail.csv')

In [66]:
df_quest_ans_mail_ans.head()

,id,questions,answers
0,1,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,хомячка....
1,2,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :).
2,3,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали..."
3,4,Эбу в двенашке называется Итэлма что за эбу? .,ЭБУ — электронный блок управления двигателем а...
4,5,академия вампиров. сколько на даный момент час...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть..."


In [67]:
#filename_mail_vec = "mail_vec.sav"
filename_mail_vec_2 = "mail_vec_2.sav"
filename_mail_tfidf = "mail_tfidf.pkl"

In [127]:
data_storage_norm = vectorize_for_annoy(df_quest_ans_mail_ans["id"], 
                                        df_quest_ans_mail_ans["questions"], 100, filename_mail_vec)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [68]:
data_storage_norm_2 = vectorize_for_annoy_2(df_quest_ans_mail_ans["id"], 
                                          df_quest_ans_mail_ans["questions"], 
                                          100, 
                                          filename_mail_vec_2, 
                                          filename_mail_tfidf)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [69]:
with open("mail_annoy_norm.pkl", 'wb') as f:
    pickle.dump(data_storage_norm_2, f)

In [71]:
NUM_TREES = 20
VEC_SIZE_EMB = 100
# index_title_emb = build_annoy(data_storage_norm, NUM_TREES, VEC_SIZE_EMB)

In [72]:
index_title_emb_2 = build_annoy(data_storage_norm_2, NUM_TREES, VEC_SIZE_EMB)

Build annoy base
computed for 1
computed for 10001
computed for 20001
computed for 30001
computed for 40001
computed for 50001
computed for 60001
computed for 70001
computed for 80001
computed for 90001
computed for 100001
computed for 110001
computed for 120001
computed for 130001
computed for 140001
computed for 150001
computed for 160001
computed for 170001
computed for 180001
computed for 190001
computed for 200001
computed for 210001
computed for 220001
computed for 230001
computed for 240001
computed for 250001
computed for 260001
computed for 270001
computed for 280001
computed for 290001
computed for 300001
computed for 310001
computed for 320001
computed for 330001
computed for 340001
computed for 350001
computed for 360001
computed for 370001
computed for 380001
computed for 390001
computed for 400001
computed for 410001
computed for 420001
computed for 430001
computed for 440001
computed for 450001
computed for 460001
computed for 470001
computed for 480001
computed for 4900

In [131]:
mail_vec_model = joblib.load(filename_mail_vec)
w2v = dict(zip(mail_vec_model.wv.index2word, mail_vec_model.wv.syn0))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [73]:
mail_vec_model_2 = joblib.load(filename_mail_vec_2)
w2v_2 = dict(zip(mail_vec_model_2.wv.index2word, mail_vec_model_2.wv.syn0))
with open(filename_mail_tfidf, 'rb') as f:
    d = pickle.load(f)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [74]:
with open("w2v_2_mail.pkl", 'wb') as f:
    pickle.dump(w2v_2, f)

In [75]:
zapros = input()

Годы правления Петра I?


In [135]:
zapros_vec = vectorize_one(zapros, 100, w2v)

In [77]:
zapros_vec_2 = vectorize_one_2(zapros, 100, w2v_2, d)

In [137]:
annoy_res = list(index_title_emb['index_title_emb'].get_nns_by_vector(zapros_vec.T, 10, include_distances=True))

In [78]:
annoy_res_2 = list(index_title_emb_2['index_title_emb']
                   .get_nns_by_vector(zapros_vec_2.T, 10, include_distances=True))

In [79]:
answer_for_mail(annoy_res, index_title_emb['map_id_2_prod_hash'], df_quest_ans_mail_ans)

NameError: name 'annoy_res' is not defined

In [81]:
answer_for_mail(annoy_res_2, index_title_emb_2['map_id_2_prod_hash'], df_quest_ans_mail_ans)

Похожий вопрос:
Годы правления Петра I? а)1631—1710 б)1672—1725 в)1721—1768. 

Ответ: 
<p> Б)1672-1725.......... </p>. 



[46536]

# Проверка загрузки модели

In [142]:
zapros = input()

О боже какой мужчина, кто поет?


In [143]:
classif_model_2 = joblib.load('logreg_for_prog_2.sav')
with open("w2v_2_for_prog.pkl", 'rb') as f:
    classif_w2v_2 = pickle.load(f)
with open("dict_tfidf_for_prog.pkl", 'rb') as f:
    d_prog = pickle.load(f)

In [144]:
pred_zapros_2(classif_model_2, zapros, size, classif_w2v_2, d_prog)

0

In [97]:
df_stack_questions = pd.read_csv('stack_questions.csv')
df_stack_answers = pd.read_csv('stack_answers.csv')

In [134]:
stack_data_storage_norm_2 = None
stack_vec_model_2 = None
VEC_SIZE_EMB = None

In [135]:
stack_VEC_SIZE_EMB = 300
stack_index_title_emb_2 = AnnoyIndex(stack_VEC_SIZE_EMB)
stack_index_title_emb_2.load('annoy_for_stack.ann')
with open("map_id_for_stack.pkl", 'rb') as f:
    stack_map_id_2_prod_hash = pickle.load(f)
with open("w2v_2_for_stack.pkl", 'rb') as f:
    stack_w2v_2 = pickle.load(f)   
with open("tfidf_for_annoy_for_stack.pkl", 'rb') as f:
    d_stack = pickle.load(f)

In [136]:
stack_zapros_vec_2 = vectorize_one_2(zapros, 300, stack_w2v_2, d_stack)

In [137]:
stack_annoy_res_2 = list(stack_index_title_emb_2.get_nns_by_vector
                         (stack_zapros_vec_2.T, 1, include_distances=True))

In [138]:
answer_for_stack(stack_annoy_res_2, 
                 stack_map_id_2_prod_hash, 
                 df_stack_questions, 
                 df_stack_answers)

Похожий вопрос:
2й максимальный элемент в массиве 

____________________________________________________________

Найден правильный ответ: 

Пример однострочного решения
Math.max.apply(Math,array.filter(function(el){return el < Math.max.apply(Math,array);}))

Суть: ищем максимальный элемент, фильтруем его, и снова ищем максимум, это и будет требуемое значение.
Что касается вашего вариант, то стоит заменить null На -Infinity начальное значение. Правда, тогда нужно будет добавить проверку в конце, чтобы вернуть null.
И еще одно замечание: использовать локальные переменные max, aftermax


var array = [-1,-2,-3,-4,-5,-5];

function main(array){
  var max = array[0];
  var afterMax = -Infinity;
  for (var i = 0; i < array.length; i++) {
    if (max < array[i]) {
      afterMax = max;
      max = array[i];
    }
    else if(max != array[i] && afterMax < array[i]){
      afterMax = array[i];
    }
  }
  console.log(afterMax === -Infinity ? null : afterMax);
}

main(array);


Еще вариант прост

[490665]

In [140]:
df_quest_ans_mail_ans = pd.read_csv('quest_ans_from_mail.csv')

In [141]:
mail_VEC_SIZE_EMB = 100
mail_index_title_emb_2 = AnnoyIndex(mail_VEC_SIZE_EMB)
mail_index_title_emb_2.load('annoy_for_mail.ann')
with open("map_id_for_mail.pkl", 'rb') as f:
    mail_map_id_2_prod_hash = pickle.load(f)
with open("w2v_2_mail.pkl", 'rb') as f:
    mail_w2v_2 = pickle.load(f)    
with open("mail_tfidf.pkl", 'rb') as f:
    d_mail = pickle.load(f)

In [145]:
mail_zapros_vec_2 = vectorize_one_2(zapros, 100, mail_w2v_2, d_mail)

In [146]:
mail_annoy_res_2 = list(mail_index_title_emb_2
                   .get_nns_by_vector(mail_zapros_vec_2.T, 10, include_distances=True))

In [147]:
answer_for_mail(mail_annoy_res_2, mail_map_id_2_prod_hash, df_quest_ans_mail_ans)

Похожий вопрос:
Почему песню, О Боже какой мужчина, поют мужчины? . 

Ответ: 
Это Вы о педерастах? А они разве мужчины? Педерасты!. 



[661058]

In [168]:
def good_answer(zapros, 
                df_stack_questions, 
                df_stack_answers, 
                df_quest_ans_mail_ans):
    ans = []
    
    classif_model_2 = joblib.load('logreg_for_prog_2.sav')
    with open("w2v_2_for_prog.pkl", 'rb') as f:
        classif_w2v_2 = pickle.load(f)
    with open("dict_tfidf_for_prog.pkl", 'rb') as f:
        d_prog = pickle.load(f)
        
    good_class = pred_zapros_2(classif_model_2, zapros, size, classif_w2v_2, d_prog)
    if good_class == -1:
        return good_class
    ans.append(good_class)
    
    if good_class == 1:
        
        stack_VEC_SIZE_EMB = 300
        stack_index_title_emb_2 = AnnoyIndex(stack_VEC_SIZE_EMB)
        stack_index_title_emb_2.load('annoy_for_stack.ann')
        with open("map_id_for_stack.pkl", 'rb') as f:
            stack_map_id_2_prod_hash = pickle.load(f)
        with open("w2v_2_for_stack.pkl", 'rb') as f:
            stack_w2v_2 = pickle.load(f)   
        with open("tfidf_for_annoy_for_stack.pkl", 'rb') as f:
            d_stack = pickle.load(f)
            
        stack_zapros_vec_2 = vectorize_one_2(zapros, 300, stack_w2v_2, d_stack)
        stack_annoy_res_2 = list(stack_index_title_emb_2
                                 .get_nns_by_vector(stack_zapros_vec_2.T, 1, include_distances=True))
        ans.append(answer_for_stack(stack_annoy_res_2, 
                         stack_map_id_2_prod_hash, 
                         df_stack_questions, 
                         df_stack_answers))
    
    elif good_class == 0:
        
        mail_VEC_SIZE_EMB = 100
        mail_index_title_emb_2 = AnnoyIndex(mail_VEC_SIZE_EMB)
        mail_index_title_emb_2.load('annoy_for_mail.ann')
        with open("map_id_for_mail.pkl", 'rb') as f:
            mail_map_id_2_prod_hash = pickle.load(f)
        with open("w2v_2_mail.pkl", 'rb') as f:
            mail_w2v_2 = pickle.load(f)    
        with open("mail_tfidf.pkl", 'rb') as f:
            d_mail = pickle.load(f)
            
        mail_zapros_vec_2 = vectorize_one_2(zapros, 100, mail_w2v_2, d_mail)
        mail_annoy_res_2 = list(mail_index_title_emb_2
                                .get_nns_by_vector(mail_zapros_vec_2.T, 10, include_distances=True))
        
        ans.append(answer_for_mail(mail_annoy_res_2, 
                                   mail_map_id_2_prod_hash, 
                                   df_quest_ans_mail_ans))
        
    return ans

In [178]:
good_answer("Молодец соси хуец", df_stack_questions, df_stack_answers, df_quest_ans_mail_ans)

[0, 56576]